### Import Required Libraries and Set Up Environment Variables

In [84]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [85]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
tmdb_api_token = os.getenv("TMDB_API_TOKEN")

### Access the New York Times API

In [86]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"


# Build URL
search_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&begin_date={begin_date}\
&end_date={end_date}&fl={field_list}&sort={sort}"

response = requests.get(search_url)
print(response)

<Response [200]>


In [49]:
# Retrieve articles
articles = requests.get(search_url).json()
articles_list = articles["response"]["docs"]

for article in articles_list:
    print(f'A snippet from the article: {article["snippet"]}')
    print('---------------------------')

A snippet from the article: A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.
---------------------------
A snippet from the article: Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.
---------------------------
A snippet from the article: Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.
---------------------------
A snippet from the article: Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.
---------------------------
A snippet from the article: A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.
---------------------------
A snippet from the article: A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock m

In [18]:


# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for pg in range(0, 20):
    # create query with a page number
    # API results show 10 articles at a time
    search_by_page_url = f"{search_url}&page={pg}"

    # Make a "GET" request and retrieve the JSON
    search_results = requests.get(search_by_page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for each_doc in search_results["response"]["docs"]:
            reviews_list.append(each_doc)
        
        # Print the page that was just retrieved
        print(f"Reviewed page number: {pg}")

        # Print the page number that had no results then break from the loop
    except:
        print(f'No results for page {pg}')
        break;

print(f'\nNumber of Reviews: {len(reviews_list)}')

Reviewed page number: 0
Reviewed page number: 1
Reviewed page number: 2
Reviewed page number: 3
Reviewed page number: 4
Reviewed page number: 5
Reviewed page number: 6
Reviewed page number: 7
Reviewed page number: 8
Reviewed page number: 9
Reviewed page number: 10
Reviewed page number: 11
Reviewed page number: 12
Reviewed page number: 13
Reviewed page number: 14
Reviewed page number: 15
Reviewed page number: 16
Reviewed page number: 17
Reviewed page number: 18
Reviewed page number: 19

Number of Reviews: 200


In [50]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[0:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [51]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [52]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# get all headline.main values to see the pattern and extract title
print(f"Number of reviews starting with Review: {len(reviews_df.loc[reviews_df['headline.main'].str.startswith('Review') == True])}")
starting_with_uni2018 = \
    len(reviews_df.loc[reviews_df["headline.main"].str.startswith("\u2018") \
                          == True])
print(f'Number of reviews starting with u2018: {starting_with_uni2018}')

# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
def get_title(str_value):
    start_index = -1
    end_index = -1
    try:
        if str_value.startswith('Review: '):
            return str_value

        start_index = str_value.index('\u2018')
        end_index = str_value.index('\u2019 Review')

        return str_value[start_index+1:end_index]
    except ValueError as ex:
        print(f"error string: {str_value}, s={start_index}, e={end_index}, with error {ex}")

reviews_df['title'] = reviews_df['headline.main'].apply(get_title)
reviews_df

Number of reviews starting with Review: 59
Number of reviews starting with u2018: 141


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Review: ‘What’s Love Got to Do With It?’ Proba...
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,Review: A Combustible Pair Find Love in ‘The O...
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,"Review: Love as the World Wars, in ‘The Ottoma..."
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Review: It’s All Mirth and Taxes in ‘Love & Ta...
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Lov

In [53]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Review: ‘What’s Love Got to Do With It?’ Proba...
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,Review: A Combustible Pair Find Love in ‘The O...
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,"Review: Love as the World Wars, in ‘The Ottoma..."
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Review: It’s All Mirth and Taxes in ‘Love & Ta...
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,No

In [54]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = reviews_df['title'].to_list()
title_list

['The Attachment Diaries',
 'Review: ‘What’s Love Got to Do With It?’ Probably a Lot',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'Th

### Access The Movie Database API

In [66]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [67]:
# Create an empty list to store the results
tmdb_movie_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in title_list:
    
    #ignore if titile is none
    if title is None:
        continue

    # Check if we need to sleep before making a request
    if (request_counter % 50) == 0:
        time.sleep(12)

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    movie_search = requests.get(url + title + tmdb_key_string).json()

    try:
        if (len(movie_search['results']) > 0):
            # Include a try clause to search for the full movie details.
            # Use the except clause to print out a statement if a movie
            # is not found.
            
            # Get movie id
            movie_id = movie_search['results'][0]['id']

            # Make a request for a the full movie details
            movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}"

            # Execute "GET" request with url
            movie_detail_response = requests.get(movie_details_url, headers=tmdb_headers).json()
            
            # Extract the genre names into a list
            genre_list = [x['name'] for x in movie_detail_response['genres']]

            # Extract the spoken_languages' English name into a list
            spoken_lang_list = [x['english_name'] for x in movie_detail_response['spoken_languages']]

            # Extract the production_countries' name into a list
            production_countries = [x['name'] for x in movie_detail_response['production_countries']]

            # Add the relevant data to a dictionary and
            # append it to the tmdb_movies_list list
            movie_detail = {
                "title" : title, 
                "original_title": movie_detail_response["original_title"], 
                "budget" :  movie_detail_response["budget"], 
                "original_language" :  movie_detail_response["original_language"], 
                "homepage" :  movie_detail_response["homepage"], 
                "overview" :  movie_detail_response["overview"], 
                "popularity" :  movie_detail_response["popularity"], 
                "runtime" :  movie_detail_response["runtime"], 
                "revenue" :  movie_detail_response["revenue"], 
                "release_date" :  movie_detail_response["release_date"], 
                "vote_average" :  movie_detail_response["vote_average"], 
                "vote_count" :  movie_detail_response["vote_count"], 
                "genres" :  genre_list,
                "spoken_languages" :  spoken_lang_list,
                "production_countries" : production_countries
            }

            tmdb_movie_list.append(movie_detail)

            # Print out the title that was found
            print(f"Found {title}")
        else:
            print(f"{title} not found.")
    except Exception as ex:
        print(f"{title} not found or errored with an exception {ex}")

The Attachment Diaries not found or errored with an exception name 'tmdb_headers' is not defined
Review: ‘What’s Love Got to Do With It?’ Probably a Lot not found.
You Can Live Forever not found or errored with an exception name 'tmdb_headers' is not defined
A Tourist’s Guide to Love not found or errored with an exception name 'tmdb_headers' is not defined
Other People’s Children not found or errored with an exception name 'tmdb_headers' is not defined
One True Loves not found or errored with an exception name 'tmdb_headers' is not defined
The Lost Weekend: A Love Story not found or errored with an exception name 'tmdb_headers' is not defined
A Thousand and One not found or errored with an exception name 'tmdb_headers' is not defined
Your Place or Mine not found or errored with an exception name 'tmdb_headers' is not defined
Love in the Time of Fentanyl not found or errored with an exception name 'tmdb_headers' is not defined
Pamela, a Love Story not found or errored with an exception 

Spelling the Dream not found or errored with an exception name 'tmdb_headers' is not defined
A Secret Love not found or errored with an exception name 'tmdb_headers' is not defined
Love Wedding Repeat not found or errored with an exception name 'tmdb_headers' is not defined
Almost Love not found or errored with an exception name 'tmdb_headers' is not defined
Hope Gap not found or errored with an exception name 'tmdb_headers' is not defined
All the Bright Places not found or errored with an exception name 'tmdb_headers' is not defined
The Photograph not found or errored with an exception name 'tmdb_headers' is not defined
You Go to My Head not found or errored with an exception name 'tmdb_headers' is not defined
Ordinary Love not found or errored with an exception name 'tmdb_headers' is not defined
To All the Boys: P.S. I Still Love You not found or errored with an exception name 'tmdb_headers' is not defined
The Woman Who Loves Giraffes not found or errored with an exception name 'tmdb

Review: In ‘The Happiest Day in the Life of Olli Maki,’ Love and Life on the Ropes not found.
Review: ‘The Promise’ Finds a Love Triangle in Constantinople not found.
Review: ‘Frantz,’ a Mysterious Frenchman and the Wounds of War not found.
Review: A Combustible Pair Find Love in ‘The Other Half’ not found.
Review: Love as the World Wars, in ‘The Ottoman Lieutenant’ not found.
Review: It’s All Mirth and Taxes in ‘Love & Taxes’ not found.
Review: ‘Everybody Loves Somebody,’ a Rom-Com With Bite not found.
Review: Cute Cats of ‘Kedi,’ Rekindling a ‘Love of Life’ not found.


In [71]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movie_list[0:5], indent=4))

[]


In [72]:
# Convert the results to a DataFrame
movies_df = pd.json_normalize(tmdb_movie_list)
print(movies_df)


Empty DataFrame
Columns: []
Index: []


### Merge and Clean the Data for Export

In [78]:
# Merge the New York Times reviews and TMDB DataFrames on title
tmdb_movies_list = []

# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

# Merge the New York Times reviews DataFrame and the TMDB DataFrame on the "title" column
merged_df = pd.merge(reviews_df, tmdb_df, how='outer', on='title')

# Print the merged DataFrame
print(merged_df)

KeyError: 'title'

In [80]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing

cols_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_fix = ['\[','\]','\'']

# Loop through the list of columns to fix
for each_col in cols_to_fix:
    # Convert the column to type 'str'
    reviews_df[each_col] = reviews_df[each_col].astype(str)

    # Loop through characters to remove
    reviews_df[each_col] = reviews_df[each_col].replace(to_replace='\[|\]|\'', value='', regex=True)

# Display the fixed DataFrame
reviews_df



KeyError: 'genres'

In [63]:
# Drop "byline.person" column
reviews_df.drop(['byline.person'], axis=1, inplace=True)
reviews_df.columns


NameError: name 'movie_reviews_df' is not defined

In [81]:
# Delete duplicate rows and reset index
reviews_df.drop_duplicates(inplace=True)
reviews_df.reset_index(drop=True)
reviews_df

TypeError: unhashable type: 'list'

In [83]:
# Export data to CSV without the index
reviews_df.to_csv("output/movie_reviews.csv")
